# Adaptive PDE discretizations on Cartesian grids
## Volume : Divergence form PDEs
## Part : Linear elasticity
## Chapter : The wave equation

We use a novel [discretization of the linear elastic energy](ElasticEnergy.ipynb) to solve the elastic wave equation, in two dimensions. The scheme is second order in space. Regarding time, we use the Verlet first order symplectic scheme, which is actually second order up to a time shift by $\mathrm{d}t/2$.

**Periodic boundary conditions.** Alternative boundary conditions will be the object of future work.

[**Summary**](Summary.ipynb) of volume Divergence form PDEs, this series of notebooks.

[**Main summary**](../Summary.ipynb) of the Adaptive Grid Discretizations 
	book of notebooks, including the other volumes.

# Table of contents
  * [1. Model analysis](#1.-Model-analysis)
    * [1.1 The Hamiltonian](#1.1-The-Hamiltonian)
    * [1.2 Choosing a material](#1.2-Choosing-a-material)
    * [1.3 Traveling waves](#1.3-Traveling-waves)
  * [2. Constant material in space](#2.-Constant-material-in-space)
    * [2.1 Isotropic material](#2.1-Isotropic-material)
    * [2.2 Crystal material (anisotropic)](#2.2-Crystal-material-(anisotropic))
  * [3. Varying material over the domain](#3.-Varying-material-over-the-domain)
    * [3.1 Smooth variations](#3.1-Smooth-variations)
    * [3.2 Discontinuous material](#3.2-Discontinuous-material)



**Acknowledgement.** Some of the experiments presented in these notebooks are part of 
ongoing research with Ludovic Métivier and Da Chen.

Copyright Jean-Marie Mirebeau, Centre Borelli, ENS Paris-Saclay, CNRS, University Paris-Saclay

## 0. Importing the required libraries

In [1]:
import sys; sys.path.insert(0,"..") # Allow import of agd from parent directory (useless if conda package installed)
#from Miscellaneous import TocTools; print(TocTools.displayTOC('ElasticityWave','Div'))

In [2]:
from agd import LinearParallel as lp
from agd import FiniteDifferences as fd
from agd.Metrics.Seismic import Hooke
from agd.ODE.hamiltonian import QuadraticHamiltonian
from agd import AutomaticDifferentiation as ad
from agd import Domain
from agd.Plotting import savefig,quiver; #savefig.dirName = 'Images/ElasticityDirichlet'
norm_infinity = ad.Optimization.norm_infinity

from agd.ExportedCode.Notebooks_Div.ElasticEnergy import ElasticEnergy
mica,ρ = Hooke.mica # Note: ρ is in g/cm^3, which is inconsistent with the Hooke tensor SI units 

In [3]:
import numpy as np; xp=np; allclose=np.allclose
import matplotlib.pyplot as plt
import scipy.sparse; import scipy.sparse.linalg
from matplotlib import rc,animation; rc('animation', html='html5')

Utility functions.

In [4]:
def quiver_animation(X,Y,U,V,subsampling=tuple(),
                     figsize=None,axis=None,quiver_kwargs={},FAkwargs={}):
    """Animates a sequence of quiver plots."""
    where = tuple(slice(None,None,s) for s in subsampling)
    X,Y,U,V = (Z.__getitem__(where) for Z in (X,Y,U,V))
    X,Y,U,V = map(ad.cupy_generic.cupy_get,(X,Y,U,V))
    fig, ax = plt.subplots(figsize=figsize); plt.close()
    if axis is not None: ax.axis(axis)
    Um,Vm = (np.mean(np.abs(E),axis=-1) for E in (U,V)) # Mean values for correct scaling
    quiv = ax.quiver(X,Y,Um,Vm,**quiver_kwargs)
    def func(i,Y): quiv.set_UVC(U[...,i],V[...,i]) 
    FAkwargs.setdefault('interval',20)
    FAkwargs.setdefault('repeat',False)
    return animation.FuncAnimation(fig,func,fargs=(Y,),frames=U.shape[-1],**FAkwargs)

### 0.1 Additional configuration

Uncomment the following line to use GPU acceleration.

In [6]:
#xp,plt,quiver,mica,allclose = map(ad.cupy_friendly,(xp,plt,quiver,mica,allclose))

Replacing numpy with cupy, set to output 32bit ints and floats by default.
Using cp.asarray(*,dtype=np.float32) as the default caster in ad.array.
Returning a copy of module matplotlib.pyplot whose functions accept cupy arrays as input.
Returning a copy of function quiver which accepts cupy arrays as input.
Replacing ndarray members with their cupy variants, for object of type <class 'agd.Metrics.Seismic.hooke.Hooke'>
Setting float32 compatible default values atol=rtol=1e-5 in np.allclose


Set to true to run large problem instances. (Computation times may become slightly longer.)

In [7]:
large_instances = xp is not np # Large instances on GPU

## 1. Model analysis


### 1.1 The Hamiltonian


The Hamiltonian of the elastic wave equation is separable, and is the sum of two quadratic terms: the kinetic energy, and the elastic energy.
$$
    H(v,m) = \frac 1 2 \int_\Omega c(\epsilon,\epsilon) + \frac{\|m\|^2}{\rho},
$$
where $c$ is the Hooke tensor, $\epsilon$ is the stress tensor, $m$ is the momentum density, and $\rho$ is the material density.
The finite differences approximation of $c(\epsilon,\epsilon)$ is the subject of [another notebook](ElasticEnergy.ipynb). In order to complete our Hamiltonian, we thus only need to discretize the kinetic energy.

In [8]:
def KineticEnergy(m,ρ):
    """Squared norm of the momentum, divided by the density, 
    which is (twice) the kinetic energy density."""
    return (m**2).sum(axis=0) / ρ

Based on the earlier discussion along the [discretization of $c(\epsilon,\epsilon)$](ElasticEnergy.ipynb), and on the [wave equation CFL condition](Time1D_Div.ipynb), we obtain a CFL condition (somewhat pessimistic) for the wave equation. 

In [9]:
def CFL(dx,hooke,ρ,order=1):
    """Largest time step guaranteed to be stable for the elastic wave equation"""
    tr = lp.trace(hooke.to_Mandel())
    return dx/(hooke.vdim*np.sqrt(tr/ρ)) # See elastic energy notebook

### 1.2 Choosing a material

We collected a few examples of material elasticity tensors and density, for illustration purposes. For simplicity, we use a constant material, and a periodic domain.

In [10]:
hooke = mica.extract_xz().rotate_by(0.5)
crystal_material = (hooke,ρ)

For the purposes of illustration, we also consider a simpler isotropic material, defined by unit Lame coefficients $\lambda=\mu=1$, and unit density $\rho=1$.

In [11]:
hooke = Hooke.from_Lame(xp.array(1.),1.)
ρ = 1.
isotropic_material = (hooke, ρ)

We are now in position to introduce the full hamiltonian, for a periodic domain at least. 

In [12]:
def WaveHamiltonian(hooke,ρ,dom):
    """Returns the Hamiltonian of the linear elastic wave equation."""
    # Summation is implicit, and purposedly not done here (for simplify_ad)
    h = dom.gridscale
    Hq = lambda q: 0.5 * ElasticEnergy(q,hooke,dom)
    Hp = lambda p: 0.5 * KineticEnergy(p,ρ)
    H = QuadraticHamiltonian(Hq,Hp)
    z = xp.zeros((dom.vdim,*dom.shape)) # Correctly shaped placeholder for position or impulsion
    H.set_spmat(z) # Replaces quadratic functions with sparse matrices
    return H 

### 1.3 Traveling waves

Traveling waves are eigenmodes of the wave equation. In the case of elastic materials, several waves of different velocities may travel in the same direction. The fastest is known as the *pressure wave*. There also exists two independent shear waves in three dimension, and a single one in two dimensions.

Traveling waves take the form
$$
    V \cos(k\cdot x - \omega t+\phi),
$$
where $k$ is the wave number, $\omega$ is the pulsation, $V$ is the amplitude, and $\phi$ is a phase shift. Given the wave vector $k$, there exists $d$ possible pulsations, and $d$ possible amplitudes (up to multiplication by a scalar).

In [13]:
def wave(t,x,ω,k,V,ϕ=0.):
    t,x,ω,k,V = fd.common_field((t,x,ω,k,V),depths=(0,1,0,1,1))
    return V*np.cos(lp.dot_VV(k,x)-ω*t+ϕ)

The `Hooke` class provides a method for computing the (normalized) amplitude and pulsation associated to a wave vector.

In [14]:
k = [0.6,0.8]
hooke,ρ = isotropic_material
ω,V = hooke.waves(k,ρ)

In dimension two, there are two wave modes. The fastest mode, associated with the largest pulsation, is the pressure wave.

In [15]:
print(f"Wave pulsations {ω} for wave vector {k}")

Wave pulsations [1.        1.7320508] for wave vector [0.6, 0.8]


In isotropic materials, the amplitude of the pressure wave is collinear with the wave vector, whereas the amplitude of shear waves is orthogonal to the wave vector.

In [16]:
print(f"Amplitude of the shear wave: {V[:,0]}, and pressure wave: {V[:,1]}")

Amplitude of the shear wave: [-0.8000001   0.59999996], and pressure wave: [0.59999996 0.8000001 ]


This property is lost in anisotropic materials. The amplitude of the shear and pressure waves is neither collinear nor orthogonal to the wave vector.

In [17]:
k = [0.6,0.8]
hooke,ρ = crystal_material
ω,V = hooke.waves(k,ρ)
print(f"Amplitude of the shear wave: {V[:,0]}, and pressure wave: {V[:,1]}, for wave vector {k}.")

Amplitude of the shear wave: [ 0.5453899 -0.8381825], and pressure wave: [-0.8381825 -0.5453899], for wave vector [0.6, 0.8].


Let us check, using automatic differentiation, the we do obtain eigenmodes of the wave equation, in the case of a constant hooke tensor and density.
This equation reads
$$
    \rho \frac{\partial^2}{\partial t^2} v_i = \sum_{j,k,l} c_{ijkl} \frac{\partial^2}{\partial x_j\partial x_k} v_l. 
$$

In [18]:
def WaveModes(k,hooke,ρ):
    """Returns the wave modes, sorted by increasing velocity, 
    associated with a given Hooke tensor and wave vector"""
    def wave_(ω,V): return lambda t,x : wave(t,x,ω,k,V)
    ω_,V_ = hooke.waves(k,ρ)
    return [wave_(ω,V) for ω,V in zip(ω_,V_.T)]

In [19]:
def WaveResidue(t,x,v,hooke,ρ):
    # Differentiate twice w.r.t time
    t_ad = ad.Dense2.identity(constant=t)
    v_tt = v(t_ad,x).hessian(0,0)
    
    # Differentiate twice w.r.t position
    x_ad = ad.Dense2.identity(constant=x)
    v_xx = v(t,x_ad).hessian() # Axes ordering : partial derivatives, then coordinates.
    
    # Contract with the Hooke tensor
    d = hooke.vdim
    hk = hooke.to_depth4() # Format hooke tensor as a depth 4 tensor
    Δv = sum(hk[:,j,k,l]*v_xx[j,k,l] for j in range(d) for k in range(d) for l in range(d))

    return ρ*v_tt - Δv

In [20]:
t,x = 1.,[2.,3.]; material = isotropic_material
shearW,pressW = WaveModes(k,*material)
assert allclose(WaveResidue(t,x,shearW,*material), 0.)
assert allclose(WaveResidue(t,x,pressW,*material), 0.)

In [21]:
t,x = 1.,[2.,3.]; material = crystal_material
shearW,pressW = WaveModes(k,*material)
assert allclose(WaveResidue(t,x,shearW,*material), 0.)
assert allclose(WaveResidue(t,x,pressW,*material), 0.,atol=1e-8 if xp is np else 1e-4)

## 2. Constant material in space

We compute elastic waves in a constant material, either isotropic or crystalline, in a periodic domain.

In [22]:
def make_domain(radius):
    """Produces the periodic domain [-radius,radius]^2, with 25 pixels per unit"""
    aX,dx = xp.linspace(-radius,radius,50*radius,endpoint=False,retstep=True)
    X=ad.array(np.meshgrid(aX,aX,indexing='ij'))
    dom = Domain.MockDirichlet(X.shape[1:],dx,padding=None) #Periodic domain (wrap instead of pad)
    return dom,X,dx

In [23]:
r = 6 if large_instances else 1 # radius to be used in numerical experiments

**Explosion and torsion initial conditions**
We use the negative gradient of a Gaussian to trigger a pressure wave in all directions, emulating an explosion.

In [24]:
def gaussian(X,σ): return np.exp(-lp.dot_VV(X,X)/(2*σ**2)) # Non normalized gaussian with prescribed variance
def explosion(X):
    """Triggers a pressure wave in all directions, emulating an explosion"""
    X_ad = ad.Dense.identity(constant=X,shape_free=(2,))
    return -gaussian(X_ad,0.1).gradient()

We display these initial conditions on small domain, otherwise the plot is hard to read. Later numerical experiments involve the large domain, if chosen so.

In [25]:
_,X,_ = make_domain(1) # small_radius = 1

In [26]:
plt.figure(figsize=(6,6)); plt.title('Explosion initial momentum'); plt.axis('equal')
quiver(*X,*explosion(X),subsampling=(2,2),scale=100.)

We rotate the explosion initial momentum by $\pi/2$ to trigger shear waves in all directions.

In [27]:
def torsion(X):
    """Triggers a torsion wave in all directions, using a torsion-like initial momentum"""
    e0,e1 = explosion(X) 
    return ad.array([-e1,e0]) # Perpendicular vector

In [28]:
plt.figure(figsize=(6,6)); plt.title("Torsion initial momentum"); plt.axis('equal')
quiver(*X,*torsion(X),subsampling=(2,2),scale=100.)

In [29]:
dom,X,dx = make_domain(r) # From now on, use the large domain

### 2.1 Isotropic material

First let us set the Hamiltonian, governing the wave equation, and the time step.

In [30]:
%%time
material = isotropic_material 
WaveH = WaveHamiltonian(*material,dom)
dt = CFL(dx,*material)

Wall time: 687 ms


**Pressure wave, and shear wave.**
We consider a pressure wave, propagating along the $x$-axis, space periodic for simplicity.

The initial condition is set as the analytic wave mode, but the time evolution is computed using the hamiltonian.

In [31]:
k = [np.pi,0.] # wave vector, 
shearW,pressW = WaveModes(k,*material)

In [32]:
%%time
q0 = pressW(0,X) # Initial displacement
t_ad = ad.Dense.identity(constant=0.)
p0 = pressW(t_ad,X).gradient(0) * 1. # Initial momentum (rho=1.)

Q,P,T = WaveH.integrate(q0,p0,scheme='Euler-p',niter=100*r,dt=dt,path=r)

Wall time: 184 ms


For these domain sizes, the computation of the propagation is quite fast, but the HTML5 encoding of the animation may take a little longer.

In [33]:
quiver_animation(*X,*Q,subsampling=(r,r),figsize=(10,10),axis='equal')

**Shear wave.** The Shear wave is slightly slower, and oscillates in the orthogonal direction, while propagating likewise along the $x$-axis.

In [34]:
%%time
q0 = shearW(0,X) # Initial displacement
t_ad = ad.Dense.identity(constant=0.)
p0 = shearW(t_ad,X).gradient(0) * 1. # Initial momentum (rho=1.)

Q,P,T = WaveH.integrate(q0,p0,scheme='Euler-p',niter=100*r,dt=dt,path=r)

Wall time: 199 ms


In [35]:
quiver_animation(*X,*Q,subsampling=(r,r),figsize=(10,10),axis='equal')

**Explosion and Torsion.**

In [36]:
%%time
q0 = np.zeros_like(X)
p0 = explosion(X)

Q,P,T = WaveH.integrate(q0,p0,scheme='Euler-p',niter=r*80,dt=dt,path=r)

Wall time: 148 ms


In [37]:
quiver_animation(*X,*Q,subsampling=(r,r),figsize=(10,10),axis='equal',quiver_kwargs={'scale':5.})

In [38]:
%%time
q0 = np.zeros_like(X)
p0 = torsion(X)

Q,P,T = WaveH.integrate(q0,p0,scheme='Euler-p',niter=r*140,dt=dt,path=r)

Wall time: 264 ms


In [39]:
quiver_animation(*X,*Q,subsampling=(r,r),figsize=(10,10),axis='equal',quiver_kwargs={'scale':5.})

### 2.2 Crystal material (anisotropic)

We reproduce the experiments of the previous section, replacing the isotropic material with a the (rotated) mica crystal.

In [40]:
%%time
material = crystal_material
WaveH = WaveHamiltonian(*material,dom)
dt = 2*CFL(dx,*material) # CFL estimation is too conservative

Wall time: 276 ms


**Pressure and shear waves.**
As before, we begin with periodic waves obtained as eigenmodes of the equation.

In [41]:
k = [np.pi,0.] 
shearW,pressW = WaveModes(k,*material)

In [42]:
%%time
q0 = pressW(0,X) # Initial displacement
t_ad = ad.Dense.identity(constant=0.)
p0 = pressW(t_ad,X).gradient(0) * 1. # Initial momentum (rho=1.)

Q,P,T = WaveH.integrate(q0,p0,scheme='Euler-p',niter=100*r,dt=dt,path=r)

Wall time: 189 ms


The oscillation direction of the pressure waves is not anymore parallel to their propagation direction.

In [43]:
quiver_animation(*X,*Q,subsampling=(r,r),figsize=(10,10),axis='equal')

In [44]:
%%time
q0 = shearW(0,X) # Initial displacement
t_ad = ad.Dense.identity(constant=0.)
p0 = shearW(t_ad,X).gradient(0) * 1. # Initial momentum (rho=1.)

Q,P,T = WaveH.integrate(q0,p0,scheme='Euler-p',niter=140*r,dt=dt,path=r)

Wall time: 275 ms


Likewise, the shear waves do not anymore oscillate in a direction orthogonal to their direction of propagation.

In [45]:
quiver_animation(*X,*Q,subsampling=(r,r),figsize=(10,10),axis='equal')

**Explosion and torsion**

In [46]:
%%time
q0 = np.zeros_like(X)
p0 = explosion(X)

Q,P,T = WaveH.integrate(q0,p0,scheme='Euler-p',niter=r*80,dt=dt,path=r)

Wall time: 150 ms


In [47]:
quiver_animation(*X,*Q,subsampling=(r,r),figsize=(10,10),axis='equal',quiver_kwargs={'scale':0.3})

In [48]:
%%time
q0 = np.zeros_like(X)
p0 = torsion(X)

Q,P,T = WaveH.integrate(q0,p0,scheme='Euler-p',niter=r*140,dt=dt,path=r)

Wall time: 300 ms


In [49]:
quiver_animation(*X,*Q,subsampling=(r,r),figsize=(10,10),axis='equal',quiver_kwargs={'scale':0.3})

## 3. Varying material over the domain

The numerical method presented here applies without modification to non-constant materials.


### 3.1 Smooth variations

We choose a Hooke tensor whose direction oscillates over the domain. 

In [50]:
hooke,ρ = crystal_material
smooth_material = hooke.rotate_by(0.3*np.sin(X[0])+0.5*np.cos(X[1])), ρ

In [51]:
material = smooth_material
WaveH = WaveHamiltonian(*material,dom)
dt = 2*CFL(dx,*material,order=2).min() # CFL estimation is too conservative

In [52]:
q0 = np.zeros_like(X)
p0 = explosion(X)

Q,P,T = WaveH.integrate(q0,p0,scheme='Euler-p',niter=2*r*140,dt=dt,path=2*r)

In [53]:
quiver_animation(*X,*Q[...,::2],subsampling=(r,r),figsize=(10,10),axis='equal',quiver_kwargs={'scale':0.3})

### 3.2 Discontinuous material

In [54]:
hooke1,ρ1 = crystal_material
hooke2,ρ2 = isotropic_material

region = X[0]<=0.4*r
ρ = np.where(region,ρ1,ρ2)

# Extract the raw hooke tensor arrays, and broadcast them
hooke1,hooke2 = (fd.as_field(e.hooke,X.shape[1:]) for e in (hooke1,hooke2))
hooke = Hooke(np.where(region,hooke1,hooke2))

discontinuous_material = hooke,ρ

In [55]:
material = discontinuous_material
WaveH = WaveHamiltonian(*material,dom)
dt = CFL(dx,*material,order=2).min() 

In [56]:
q0 = np.zeros_like(X)
p0 = explosion(X)

Q,P,T = WaveH.integrate(q0,p0,scheme='Euler-p',niter=2*r*160,dt=dt,path=2*r)

In [57]:
quiver_animation(*X,*Q[...,::2],subsampling=(r,r),figsize=(10,10),axis='equal',quiver_kwargs={'scale':0.3})